# Demo of iteration options

In [1]:
import datetime as dt
import pandas as pds

In [2]:
import pysat

In [3]:
# Set data directory if user hasn't already set one
print(f"old: {pysat.params['data_dirs']}")
if len(pysat.params['data_dirs']) == 0 or pysat.params['data_dirs'] == ['.']:
    # Set a directory for pysat to use for data
    pysat.params['data_dirs'] = '/home/jovyan/scratch_space/.pysat/'
else:
    print('pysat directory has been set previously. Leaving unchanged.')

print(f"new: {pysat.params['data_dirs']}")

old: ['/home/jovyan/scratch_space/.pysat']
pysat directory has been set previously. Leaving unchanged.
new: ['/home/jovyan/scratch_space/.pysat']


In [4]:
# All of the examples work with any of the Instrument types below.

# Create a custom range of files supported by test instruments
start_range = dt.datetime(2022, 1, 1)
stop_range = dt.datetime(2022, 12, 31)
file_date_range = pds.date_range(start_range, stop_range, freq='D')

# Set orbit iteration parameters. Possible values for 'kind' are
# 'local time', 'longitude', 'polar', or 'orbit'. 
#         - local time: negative gradients in lt or breaks in inst.index
#         - longitude: negative gradients or breaks in inst.index
#         - polar: zero crossings in latitude or breaks in inst.index
#         - orbit: uses unique values of orbit number
# 'index' maps to the variable used for iteration.
# 'period' applys orbit breakdown filtering based upon the provided orbit period.
# Currently only applies to 'local time' orbits.
orbit_info = {'kind': 'lt', 'index': 'mlt', 
              'period': dt.timedelta(minutes=95)}

# Collect instantation keywords
kwargs = {'file_date_range': file_date_range,
          'orbit_info': orbit_info}

# Select one of the test instrument below by uncommenting one of the lines

# Basic test instrument, pandas format
inst = pysat.Instrument('pysat', 'testing', **kwargs)

# Basic test instrument, xarray format
# inst = pysat.Instrument('pysat', 'testing_xarray', **kwargs)

# Test instrument, higher dimensional data, xarray format
# inst = pysat.Instrument('pysat', 'testing2d_xarray', **kwargs)

# Basic test model, xarray format
# inst = pysat.Instrument('pysat', 'testmodel', tag='pressure_levels', **kwargs)

In [5]:
# Set bounds to iterate over. Accepts information for the 
# start, stop, step size, and loaded data width when iterating.
# Users can supply either a single start, stop, or a list of starts/stops.

# Iteration works either with datetimes/timestamps or with filenames.

start = start_range + pds.DateOffset(weeks=0)
stop = start_range + pds.DateOffset(days=13)

# Demonstrate assigning multiplate date ranges
# start = [start, start + pds.DateOffset(months=3)]
# stop = [stop, stop + pds.DateOffset(months=3)]

# Number of days to move forward for each subsequent load.
# Uses pandas frequency strings. '3D' is three days.
step = '3D'

# Number of days to load for each iteration. In general, `step` and `width` 
# don't have to be the same.
width = dt.timedelta(days=3)

In [6]:
# Assign iteration limits
inst.bounds = (start, stop, step, width)

In [7]:
# Loop through data loading `width` days each loop, and moving forward `step` for the subsequent load.
for loop_inst in inst:
    print('Loaded data range: ', repr(loop_inst.index[0]), 
          repr(loop_inst.index[-1]))

Loaded data range:  Timestamp('2022-01-01 00:00:00') Timestamp('2022-01-03 23:59:59')
Loaded data range:  Timestamp('2022-01-04 00:00:00') Timestamp('2022-01-06 23:59:59')
Loaded data range:  Timestamp('2022-01-07 00:00:00') Timestamp('2022-01-09 23:59:59')
Loaded data range:  Timestamp('2022-01-10 00:00:00') Timestamp('2022-01-12 23:59:59')


In [8]:
# Same limits are applied when iterating by orbit. Note that the `step` and `width` must have the same value.
# Note that orbits will start/stop across daybreaks. The exceptions to this are the first/last day
# of bounds as the previous/next day can't be loaded.

# Load first day of data to ensure we start at begining. This is here since the iteration above will result
# in `inst` having loaded data for the last day.
inst.load(date=inst.bounds[0][0])

for loop_inst in inst.orbits:
    print('Loaded data range: ', repr(loop_inst.index[0]), 
          repr(loop_inst.index[-1]))    

Loaded data range:  Timestamp('2022-01-01 00:00:00') Timestamp('2022-01-01 00:21:59')
Loaded data range:  Timestamp('2022-01-01 00:22:00') Timestamp('2022-01-01 01:58:59')
Loaded data range:  Timestamp('2022-01-01 01:59:00') Timestamp('2022-01-01 03:35:59')
Loaded data range:  Timestamp('2022-01-01 03:36:00') Timestamp('2022-01-01 05:12:59')
Loaded data range:  Timestamp('2022-01-01 05:13:00') Timestamp('2022-01-01 06:49:59')
Loaded data range:  Timestamp('2022-01-01 06:50:00') Timestamp('2022-01-01 08:26:59')
Loaded data range:  Timestamp('2022-01-01 08:27:00') Timestamp('2022-01-01 10:03:59')
Loaded data range:  Timestamp('2022-01-01 10:04:00') Timestamp('2022-01-01 11:40:59')
Loaded data range:  Timestamp('2022-01-01 11:41:00') Timestamp('2022-01-01 13:17:59')
Loaded data range:  Timestamp('2022-01-01 13:18:00') Timestamp('2022-01-01 14:54:59')
Loaded data range:  Timestamp('2022-01-01 14:55:00') Timestamp('2022-01-01 16:31:59')
Loaded data range:  Timestamp('2022-01-01 16:32:00') T